# LJ potential analyic derivatives derivation

Sympy-based derivation of LJ-backward analytic form. Create sympy definitions of each segment of the LJ potential, then emit simplified/cse-eliminated forms of each segment. Segemnt functional forms are copied, with light editing, to lj.hh potential.

## Utilities

Simple utility function to emit simplied/cse-elimianted C++ code for a given sympy expression.

In [51]:
import jinja2
import sympy
import sympy.printing

def format_expression(name, full_expression):
    subterms, expr = sympy.cse(full_expression.simplify())

    template = jinja2.Template("""
    {% if subterms %}
    {% for t_name, t_code in subterms %}
    Real {{sympy.printing.cxxcode(t_code, assign_to=t_name)}}{% endfor %}
    {% endif %}

    {{sympy.printing.cxxcode(expr[0], assign_to=name)}}

    """)

    return template.render(sympy=sympy, subterms = subterms, expr = expr, name=name)

## Symbolic Variables

Define symbolic variables for inputs and parameters than will be used in potential segments.

In [52]:
(
    dist,
    lj_sigma,
    lj_switch_slope,
    lj_switch_intercept,
    lj_coeff_sigma12,
    lj_coeff_sigma6,
    lj_spline_y0,
    lj_spline_dy0,
    lj_switch_dis2sigma,
    spline_start,
    max_dis,
) = sympy.symbols("""
    dist,
    lj_sigma,
    lj_switch_slope,
    lj_switch_intercept,
    lj_coeff_sigma12,
    lj_coeff_sigma6,
    lj_spline_y0,
    lj_spline_dy0,
    lj_switch_dis2sigma,
    spline_start,
    max_dis,
""")

## Piecewise Potential
Piecewise definition of the potential over spline, 6-12, and linear segments.

In [53]:
def lr_spline_fade():
    x0 = spline_start
    x1 = max_dis

    x = dist
    y0 = lj_spline_y0
    dy0 = lj_spline_dy0
    u0 = (3.0 / (x1 - x0)) * ((-y0) / (x1 - x0) - dy0)
    u1 = (3.0 / (x1 - x0)) * (y0 / (x1 - x0))

    return ((x - x1) * ((x - x0) * (u1 * (x0 - x) + u0 * (x - x1)) + 3.0 * y0)) / (
        3.0 * (x0 - x1)
    )


def analytic():
    invdist2 = 1.0 / (dist * dist)
    invdist6 = invdist2 * invdist2 * invdist2
    invdist12 = invdist6 * invdist6

    return (lj_coeff_sigma12 * invdist12) + (lj_coeff_sigma6 * invdist6)


def linear():
    return dist * lj_switch_slope + lj_switch_intercept


## Analyic Derivatives

### Long-range spline fade

In [57]:
print(
    format_expression( "d_lj_d_dist", lr_spline_fade().diff(dist))
)


    
    
    Real x0 = -spline_start;
    Real x1 = max_dis + x0;
    Real x2 = dist + x0;
    Real x3 = lj_spline_y0*x2;
    Real x4 = dist - max_dis;
    Real x5 = lj_spline_dy0*x1;
    Real x6 = x4*(lj_spline_y0 + x5);
    

    d_lj_d_dist = 0.33333333333333331*(-3.0*lj_spline_y0*std::pow(x1, 2) + 3.0*x2*(x3 + x6) + x4*(x2*(6.0*lj_spline_y0 + 3.0*x5) + 3.0*x3 + 3.0*x6))/std::pow(x1, 3);

    


### Primary 6-12

In [58]:
print(
    format_expression( "d_lj_d_dist", analytic().diff(dist))
)


    

    d_lj_d_dist = -(6.0*std::pow(dist, 6)*lj_coeff_sigma6 + 12.0*lj_coeff_sigma12)/std::pow(dist, 13);

    


### Short-range linear repulsive

In [59]:
print(
    format_expression( "d_lj_d_dist", linear().diff(dist))
)


    

    d_lj_d_dist = lj_switch_slope;

    
